In [1]:
import sys

print(sys.version)

3.8.10 (default, Nov 14 2021, 21:32:59) 
[Clang 12.0.5 (clang-1205.0.22.9)]


In [2]:
SPARK_HOME = "/Users/kun/github/spark/spark-3.1.2-bin-hadoop-3.2.2"

In [3]:
import findspark

findspark.init(SPARK_HOME)
#findspark.add_packages(["org.apache.hadoop:hadoop-aws:3.2.2", "com.amazonaws:aws-java-sdk-bundle:1.11.375"])

### Spark Session 생성

로컬모드에서 실행할 Spark Session 을 만듭니다. (`.master("local[*]")`)
- 일반적인 Spark 설정은 `$SPARK_HOME/conf/spark-defaults.conf` 내에서 세팅해 공통환경으로 사용합니다. 다만 이 예제에서는 보여주기 위해 SparkConf 를 이용해 설정합니다.
- Hive Metastore URI 등 HMS 관련 설정은 `$SPARK_HOME/conf/hive-site.conf` 내에서 세팅해 공통 환경으로 사용합니다.
- 이 예제에서는 Minio 를 사용하므로 Access Key, Secret Key 를 사용합니다. AWS 위에서 실행된다면 [AWS Instance Profile](https://docs.aws.amazon.com/ko_kr/IAM/latest/UserGuide/id_roles_use_switch-role-ec2_instance-profiles.html) 을 이용할 수 있으므로 키를 세팅하지 않습니다.

In [4]:
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("example-app") \
    .config("spark.hadoop.fs.s3a.access.key", "accesskey")\
    .config("spark.hadoop.fs.s3a.secret.key", "secretkey")\
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled","false")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .enableHiveSupport() \
    .getOrCreate()
    
spark.sparkContext.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")

21/11/29 15:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/29 15:41:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
spark.sparkContext.getConf().getAll()

In [7]:
spark.sql("""
CREATE TABLE student (
    id INT, 
    name STRING, 
    age INT
) 
STORED AS PARQUET
LOCATION 's3a://udon-data/lake/student/'
""")

21/11/26 01:44:27 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [17]:
df = spark.read.format("csv").load("s3a://udon-data-lake/marketing_campaign.csv")